<a href="https://colab.research.google.com/github/Suyog-J/Plant_pathogen_tests/blob/main/pathogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
! kaggle datasets download -d sujallimje/plant-pathogens

Dataset URL: https://www.kaggle.com/datasets/sujallimje/plant-pathogens
License(s): CC-BY-NC-SA-4.0
100% 23.7G/23.7G [05:30<00:00, 113MB/s]
100% 23.7G/23.7G [05:30<00:00, 76.9MB/s]


In [3]:
import os

In [ ]:
!unzip /content/plant-pathogens.zip

Streaming output truncated to the last 5000 lines.
  inflating: pathogen/fungus/fung (12936).JPG  
  inflating: pathogen/fungus/fung (12937).JPG  
  inflating: pathogen/fungus/fung (12938).JPG  
  inflating: pathogen/fungus/fung (12939).JPG  
  inflating: pathogen/fungus/fung (1294).JPG  
  inflating: pathogen/fungus/fung (12940).JPG  
  inflating: pathogen/fungus/fung (12941).JPG  
  inflating: pathogen/fungus/fung (12942).JPG  
  inflating: pathogen/fungus/fung (12943).JPG  
  inflating: pathogen/fungus/fung (12944).JPG  
  inflating: pathogen/fungus/fung (12945).JPG  
  inflating: pathogen/fungus/fung (12946).JPG  
  inflating: pathogen/fungus/fung (12947).JPG  
  inflating: pathogen/fungus/fung (12948).JPG  
  inflating: pathogen/fungus/fung (12949).JPG  
  inflating: pathogen/fungus/fung (1295).JPG  
  inflating: pathogen/fungus/fung (12950).JPG  
  inflating: pathogen/fungus/fung (12951).JPG  
  inflating: pathogen/fungus/fung (12952).JPG  
  inflating: pathogen/fungus/fung (1295

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import pandas as pd, numpy as np

In [ ]:
img_size = 256
BATCH_SIZE = 150
Channels = 3
EPOCHS = 5

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'pathogen/',
    shuffle = True,
    image_size = (img_size,img_size),
    batch_size = BATCH_SIZE

)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
plt.figure(figsize=(10,10))
for image_batch, label_batch in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(image_batch[i].numpy().astype('int'))
    plt.title(class_names[label_batch[i]])
    plt.axis('off')

In [ ]:
def split_train_test(data, train_split = 0.8, val_split = 0.1, test_split = 0.1, shuffle = True, shuffle_size = 10000):
  data_size = len(data)

  if shuffle:
    data = data.shuffle(shuffle_size, seed = 12)

  train_size = int(data_size*train_split)
  val_size = int(data_size*val_split)

  train_ds = data.take(train_size)

  val_ds = data.skip(train_size).take(val_size)
  test_ds = data.skip(train_size).skip(val_size)
  return train_ds, val_ds, test_ds

In [ ]:
train_ds,val_ds,test_ds = split_train_test(dataset)

In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
len(test_ds)

In [ ]:
# cache and prefetch
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size= tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
# rescale and resize
resize_rescale = tf.keras.Sequential([
    layers.Resizing(img_size,img_size),
    layers.Rescaling(1.0/255)
])

In [ ]:
# data augumentation
data_agumentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomRotation(0.2)
])

In [ ]:
# Deep neural network
input_shape = [BATCH_SIZE,img_size,img_size,Channels]
n_classes = 5
model = models.Sequential([
    resize_rescale,
    data_agumentation,
    layers.Conv2D(32, kernel_size=(3,3), activation= 'relu', input_shape = input_shape),#1
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),#1
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),#2
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size =(3,3), activation='relu'),#3
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size =(3,3), activation='relu'),#4
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size =(3,3), activation='relu'),#5
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation= 'relu'),
    layers.Dense(n_classes, activation= 'softmax')
])

model.build(input_shape = input_shape)

In [ ]:
model.summary()

In [ ]:
# model compile
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [ ]:
# model fit
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    verbose = 1,
    batch_size = BATCH_SIZE,
    validation_data = val_ds
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
history.params

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS), acc, label = 'Training accuracy' )
plt.plot(range(EPOCHS), val_acc, label = 'Valadition accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS), loss , label = 'Training loss')
plt.plot(range(EPOCHS), val_loss, label = 'Validation loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validiation loss')

In [ ]:
for images_batch, labels_batch in test_ds.take(1):
  first_image = plt.imshow(image_batch[0].numpy().astype('int'))
  first_label = class_names[labels_batch[0].numpy()]

  print('first image predicted', first_image)
  print('first image actual image : ', first_label)

  batch_prediction = model.predict(image_batch)
  print('predicted label : ',class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predicts(model, img):
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array,0) # create batch

  prediction = model.predict(img_array)
  predicted_class = class_names[np.argmax(prediction[0])]
  confidence = round(100*(np.max(prediction[0])),2)
  return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15,15))
for image, labels in test_ds.take(1):
  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(image[i].numpy().astype('int'))

    predicted_class, confidence = predicts(model,image[i].numpy())
    actual_class = class_names[labels[i]]
    plt.title(f'Actual class : {actual_class} \n Predicted class : {predicted_class}\n Confidence : {confidence}%')
    plt.axis('off')